# Analysis for the Hydrogen Laser Spectroscopy Experiment

In [1]:
# Run this cell with Shift-Enter, and wait until the asterisk changes to a number, i.e., [*] becomes [1]
import numpy as np
import scipy.constants as const
import uncertainties as unc
import uncertainties.unumpy as up
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
# Set up plot defaults
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 11.0,8.0  # Roughly 11 cm wde by 8 cm high
mpl.rcParams['font.size'] = 12.0 # Use 12 point font

## Summary of tasks

### Prior to working on this notebook

1. Work out exercises 1, 2 and 10.  These concern the Balmer-$\alpha$ wavelength, the term diagram of the $n=2$ to $n=3$ transitions and selection rules, and the overall layout of Doppler-free beam-crosssing method of laser spectrosopy.


2. Obtain chart-recorder scans for the two hydrogen spectrum data runs (one for "left-to-right" scan, i.e., decreasing wavelength, the other for "right-to-left", or increasing wavelength).


3. Extract from each scan the positions in scan units (voltage or paper grid) of (1) the peaks in the Fabry-Perot (F-P) mirror output, and (2) positions of the 5 obvious peaks in the probe-beam measurement. Positions should be estimated to at least three significant figures in the measurement unit.  The F-P peak positions do not need uncertainties, but you should assign an uncertainty to the prob-beam peak positions.


4. Obtain the Fabry-Perot mirror separation measurements.  You should have 6-8 separate measurements to average.


5. Create spreadsheets with these data to be read into the notebook for further processing.  Recommended: one .xlsx file with 5 tabs (worksheets), or 5 separate .csv files, one for the mirror measurements and one for each set of peak position measurements.  Read them in with either Pandas `read_csv()` or `read_excel()`.


### Tasks for this notebook

Convert the mirror separation measurements to meters and calculate the separation with its uncertainty ("standard deviation of the mean").  Then calculate the free spectral range in MHz with uncertainty. 

For each scan:

<blockquote>
Plot the F-P peak index (y-axis) versus its position (x-axis).  Fit this to a 3rd-order polynomial to produce calibration coefficients.

Create a calibration function that takes XY recorder positions and returns position in FSR units (relative to an arbitrary origin).

Use the calibration function to convert probe-beam peak positions to positions in FSR units. Incorporate the uncertainty in the probe-beam peak positions to get uncertainty in FSR units.  

Convert the FSR units to frequency using the free spectral range, including uncertaities.

Finally, calculate the frequency separations requested, among them the "Lamb shift" which is the difference between the 2S<sub>1/2</sub>-3P<sub>3/2</sub> and 2P<sub>1/2</sub>-3D<sub>3/2</sub> transitions.
</blockquote>

Once the above has been completed for both scans, combine the results to obtain the the best values for the fine-structure frequency separations, and compare the measurements to the expected values.  (these can be found in the experiment instructions.)

## Read in the hydrogen spectrum data

In [3]:
# Read in the data
Hdata = pd.read_excel("Laser Spectroscopy Data.xlsx", sheet_name = None)
Hdata

{'F-P Data':    Distance1(mm)  Distance2(mm)  Distance3(mm)  Distance4(mm)  Distance5(mm)  \
 0          99.25           96.2           96.2           96.8           99.1   
 
    Average(mm)  StandardDeviation(mm)  Mirror1Recess(mm)  Mirror2Recess(mm)  
 0        97.51               1.540454              0.015              0.005  ,
 'Down Probe':    Peak   Peak(V)  PeakLowerUnc(V)  PeakUpperUnc(V)
 0     1 -0.735607        -0.744638        -0.724882
 1     2 -0.191773        -0.201498        -0.185289
 2     3 -0.032209        -0.039773        -0.023924
 3     4  0.131025         0.122672         0.142088
 4     5  0.240301         0.230367         0.251590,
 'Up Probe':    Peak   Peak(V)  PeakLowerUnc(V)  PeakUpperUnc(V)
 0     1 -0.529370        -0.540884        -0.516952
 1     2  0.069289         0.062844         0.074123
 2     3  0.230427         0.223981         0.240095
 3     4  0.387770         0.380151         0.400470
 4     5  0.490499         0.482032         0.500094,
 

In [4]:
# Display
Hdata["Down Pump"]

,Peak,Peak(V),PeakLowerUnc(V),PeakUpperUnc(V)
0,1,-1.101706,-1.130711,-1.079147
1,2,-0.947014,-0.977630,-0.926066
2,3,-0.800379,-0.822938,-0.785877
3,4,-0.663412,-0.687583,-0.636019
4,5,-0.513555,-0.539336,-0.489384
5,6,-0.368531,-0.391090,-0.341137
6,7,-0.217062,-0.238009,-0.194502
7,8,-0.076872,-0.101043,-0.051090
8,9,0.076209,0.053649,0.098768
9,10,0.235735,0.208341,0.261517


##  Mirror separation & Free spectral range

Calculate the mean and standard deviation of the mean and save in an uncertainty object.

Calculate the free spectral range in Hz (with uncertainty).  The free spectral range is 

$$ FSR = \frac{c}{2d}$$

In [5]:
# Show a table of the mirror separation values
Hdata["F-P Data"]


,Distance1(mm),Distance2(mm),Distance3(mm),Distance4(mm),Distance5(mm),Average(mm),StandardDeviation(mm),Mirror1Recess(mm),Mirror2Recess(mm)
0,99.25,96.2,96.2,96.8,99.1,97.51,1.540454,0.015,0.005


In [6]:
# Calculate the mean, standard deviation, and standard deviation of the mean 
# of the mirror separation values in meters.

#Use recess data to adjust measurements for it
MRecess = Hdata["F-P Data"]["Mirror1Recess(mm)"][0] + Hdata["F-P Data"]["Mirror1Recess(mm)"][0]

#Convert to meters
FP_sep = (Hdata["F-P Data"]["Average(mm)"][0] + MRecess) * 10**-3

FP_sep_stderr = np.std(Hdata["F-P Data"].iloc[0,:5]) * 10**-3

# Build an uncertainty object to hold the mirror separation with its uncertainty.
# Print the mirrr separation and uncertainty in cm.

uFP_sep = unc.ufloat(FP_sep, FP_sep_stderr)
print('Fabry-Perot mirror separation = {:.1uP} cm'.format(uFP_sep/const.centi))

# Print the free spectral range in MHz

uFSR = const.c/(2*uFP_sep)
print('F-P set free spectral range = {:.1uP} MHz'.format(uFSR/const.mega))

Fabry-Perot mirror separation = 9.8±0.1 cm
F-P set free spectral range = (1.54±0.02)×10³ MHz


## Left-to-right scan analysis

### Fit the FP peaks

First, make a table of the FP peak locations.

Create an array of **indices** of the F-P peak locations. 

Fit the index array (y-axis) vs. peak locations (x-axis) to a 3rd-order polynomial to obtain calibration constants. 

In [7]:
# The indices of the Pandas DataFrame are given by the attribute 'index'
# To get an array with these indices, make a Python list, e.g., 'list(DataFrame.index)'

#Use up pump data to setup the indices
indices = list(Hdata["Up Pump"].index)


Then set up the fitting.

In [8]:
# Use a polynmial of degree=3
from lmfit.models import PolynomialModel

curve = PolynomialModel(degree=3)

# The return is the fit parameters structure

def poly_fit_and_plot(xdata, ydata, yerr=None, model=PolynomialModel(degree=3), xlabel='X', ylabel='Y', show=True):
    '''
    Fit a line or curve, and plot/show the fit results.
    The function returns a parameters object with the fit parameters
    '''
    param_guess = model.guess(ydata, x=xdata)
    if (yerr is None):
        model_fit = model.fit(ydata, param_guess, x=xdata)
    else:
        model_fit = model.fit(ydata, param_guess, x=xdata, weights=1/yerr)
    if show==True:
        print(model_fit.fit_report(show_correl=False))
        model_fit.plot()
        plt.xlabel(xlabel)
        plt.ylabel(ylabel);

    return model_fit.params

Execute the fitting.

In [9]:
# Carry out the fit, show the results, and save the fit parameters
PeakUnc_up = abs((Hdata["Up Pump"]["PeakLowerUnc(V)"] - Hdata["Up Pump"]["PeakUpperUnc(V)"]))

upPump_params = poly_fit_and_plot(Hdata["Up Pump"]["Peak(V)"], indices, yerr = PeakUnc_up,
                                 xlabel = "Fabry-Perot Peak Locations L-R (V)", ylabel = "Peak Index")

[[Model]]
    Model(polynomial)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 11
    # data points      = 11
    # variables        = 4
    chi-square         = 0.97818824
    reduced chi-square = 0.13974118
    Akaike info crit   = -18.6194327
    Bayesian info crit = -17.0278516
    R-squared          = 0.99997855
[[Variables]]
    c0:  5.17681666 +/- 0.00834359 (0.16%) (init = 5.176902)
    c1:  6.48488877 +/- 0.02868969 (0.44%) (init = 6.488112)
    c2:  0.70101488 +/- 0.02789660 (3.98%) (init = 0.6971043)
    c3: -0.07098425 +/- 0.05940504 (83.69%) (init = -0.07986113)


### Make the calibration function

You can use the same method to make the calibration curve as you may have used in the Zeeman effect experiment or Franck-Hertz experiment, e.g., use `eval()` with the fit coefficients.

In [10]:
# Define a calibration function

# Start by defining a function which obstains the coefficient
def get_ucoeff(params,coeff):
    return unc.ufloat(params[coeff].value, params[coeff].stderr)
    
def cal(xdata, fit_params):
  # Fill in function
    c0 = get_ucoeff(fit_params, "c0")
    c1 = get_ucoeff(fit_params, "c1")
    c2 = get_ucoeff(fit_params, "c2")
    c3 = get_ucoeff(fit_params, "c3")
    ydata = c3 * (xdata**3) + c2*(xdata**2) + c1 * xdata + c0

    return ydata
    
# Test
print(cal(Hdata["Up Probe"]["Peak(V)"], upPump_params))

0    1.951+/-0.021
1    5.629+/-0.009
2    6.707+/-0.011
3    7.793+/-0.015
4    8.518+/-0.019
Name: Peak(V), dtype: object


### Calculate the spectrum 

In [11]:
# Make uncertainty array of probe-beam paper-unit locations with uncertainty


ProbeUp_peaks = up.uarray(Hdata["Up Probe"]["Peak(V)"],
                          abs((Hdata["Up Probe"]["PeakLowerUnc(V)"] - Hdata["Up Probe"]["PeakUpperUnc(V)"])/2))

#FSR array
FSR_array = cal(ProbeUp_peaks, upPump_params)

#Use to find the mHz array
mHz_array = FSR_array*(uFSR/const.mega)

#Find the relative distance
rel_mHz = mHz_array - mHz_array[0]

# Build Pandas DataFrame with spectrum in paper units, FSR units (applying calibration), 
# MHz (applying free spectral range), and MHz relative to the largest, lowest-frequency peak.

Left_Right_spectrum = pd.DataFrame({'Paper': ProbeUp_peaks,
                                    'FSR': FSR_array,
                                    'MHz': mHz_array,
                                    'Rel MHz': rel_mHz
                                   })

# Show the table
print('Left-to-right scan probe-beam peak locations:')
Left_Right_spectrum

Left-to-right scan probe-beam peak locations:


,Paper,FSR,MHz,Rel MHz
0,-0.529+/-0.012,1.95+/-0.07,(3.00+/-0.12)e+03,0.0+/-0
1,0.069+/-0.006,5.63+/-0.04,(8.65+/-0.14)e+03,(5.65+/-0.15)e+03
2,0.230+/-0.008,6.71+/-0.06,(1.031+/-0.017)e+04,(7.31+/-0.17)e+03
3,0.388+/-0.010,7.79+/-0.07,(1.198+/-0.020)e+04,(8.98+/-0.20)e+03
4,0.490+/-0.009,8.52+/-0.07,(1.309+/-0.021)e+04,(1.009+/-0.021)e+04


### Calculate separations

Make a table that shows the following separations in MHz (with uncertainty):

* The Lamb shift: 2S<sub>1/2</sub> - 3P<sub>3/2</sub> to 2P<sub>1/2</sub> - 3D<sub>3/2</sub> frequency separation (with uncertainty) for both scans.
* The 2P<sub>3/2</sub>  - 3D<sub>5/2</sub> to 2P<sub>1/2</sub> - 3D<sub>3/2</sub>  frequency separation, i.e., highest frequency peak to lowest frequency peak (with uncertainty) for both scans.
* The 2P<sub>3/2</sub>  - 3D<sub>5/2</sub> to 2S<sub>1/2</sub> - 3P<sub>1/2</sub> frequency separation, i.e., tallest to smallest peak (with uncertaintyl) for both scans.


In [12]:
# Calculate the separations and make a table.
# Recommended: Build a DataFrame.  You can use it to add the calculated separations from the 
# other scan below and then the averages of both to compare to accepted results.

    
# Here is a list of the separation labels
seps_list = ['2S1/2-3P3/2 to 2P1/2-3D3/2', '2P3/2-3D5/2 to 2P1/2-3D3/2','2P3/2-3D5/2 to 2S1/2-3P1/2']

# Make a list of the separations in MHz
LR_list = [rel_mHz[4] - rel_mHz[3], rel_mHz[4] - rel_mHz[0], rel_mHz[1] - rel_mHz[0]]

# Construct the DataFrame
Results = pd.DataFrame({'Separations': seps_list,
                            'L-R (MHz)':LR_list})

# Display
Results

,Separations,L-R (MHz)
0,2S1/2-3P3/2 to 2P1/2-3D3/2,(1.11+/-0.15)e+03
1,2P3/2-3D5/2 to 2P1/2-3D3/2,(1.009+/-0.021)e+04
2,2P3/2-3D5/2 to 2S1/2-3P1/2,(5.65+/-0.15)e+03


## Right-to-left scan analysis
### Fit the F-P peaks


In [13]:
# Make a table of the F-P peaks, and index array
PeakUnc_down = abs((Hdata["Down Pump"]["PeakLowerUnc(V)"] - Hdata["Down Pump"]["PeakUpperUnc(V)"]))
indices


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
# Fit to a polynomial of degree=3, and show the results and save the parameters
downPump_params = poly_fit_and_plot(Hdata["Down Pump"]["Peak(V)"], indices, yerr = PeakUnc_down,
                                 xlabel = "Fabry-Perot Peak Locations Down(V)", ylabel = "Peak Index")


[[Model]]
    Model(polynomial)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 11
    # data points      = 11
    # variables        = 4
    chi-square         = 2.16884152
    reduced chi-square = 0.30983450
    Akaike info crit   = -9.86072320
    Bayesian info crit = -8.26914211
    R-squared          = 0.99995340
[[Variables]]
    c0:  7.49005577 +/- 0.01441538 (0.19%) (init = 7.489739)
    c1:  6.64703358 +/- 0.03562930 (0.54%) (init = 6.639523)
    c2: -0.46481465 +/- 0.11681840 (25.13%) (init = -0.4520077)
    c3: -0.28460475 +/- 0.10237814 (35.97%) (init = -0.2673827)


### Make the calibration for the R-L scan


In [15]:
# Define and test the calibration function
#Use the function from before
print(cal(Hdata["Down Probe"]["Peak(V)"], downPump_params))


0      2.46+/-0.08
1    6.200+/-0.017
2    7.275+/-0.014
3    8.352+/-0.015
4    9.057+/-0.018
Name: Peak(V), dtype: object


### Calculate the spectrum

Carry out the calculation and table construction for the right-to-left scan.

In [16]:
# Make uncertainty array of probe-beam paper-unit locations with uncertainty

ProbeDown_peaks = up.uarray(Hdata["Down Probe"]["Peak(V)"],
                          abs((Hdata["Down Probe"]["PeakLowerUnc(V)"] - Hdata["Down Probe"]["PeakUpperUnc(V)"])/2))

#FSR array
FSR_array_down = cal(ProbeDown_peaks, downPump_params)

#Use to find the mHz array
mHz_array_down = FSR_array_down*(uFSR/const.mega)

#Find the relative distance
rel_mHz_down = mHz_array_down - mHz_array_down[0]

# Build Pandas DataFrame with spectrum in paper units, FSR units (applying calibration), 

Right_left_spectrum = pd.DataFrame({'Paper': ProbeDown_peaks,
                                    'FSR': FSR_array_down,
                                    'MHz': mHz_array_down,
                                    'Rel MHz': rel_mHz_down
                                   })

# Show the table
print('Right-to-left scan probe-beam peak locations:')
Right_left_spectrum

Right-to-left scan probe-beam peak locations:


,Paper,FSR,MHz,Rel MHz
0,-0.736+/-0.010,2.46+/-0.11,(3.78+/-0.17)e+03,0.0+/-0
1,-0.192+/-0.008,6.20+/-0.06,(9.53+/-0.16)e+03,(5.74+/-0.19)e+03
2,-0.032+/-0.008,7.28+/-0.05,(1.118+/-0.018)e+04,(7.40+/-0.21)e+03
3,0.131+/-0.010,8.35+/-0.07,(1.284+/-0.021)e+04,(9.05+/-0.23)e+03
4,0.240+/-0.011,9.06+/-0.07,(1.392+/-0.022)e+04,(1.013+/-0.024)e+04


### Calculate separations

Calculate the separations for the right-to-left scan.

In [17]:
# Calculate the separations in the R to L scan, and add a column to the data frame 
# for separation measurements

#Seperations from right to left
RL_list = [rel_mHz_down[4] - rel_mHz_down[3], rel_mHz_down[4] - rel_mHz_down[0], rel_mHz_down[1] - rel_mHz_down[0]]

# Construct the DataFrame
Results_RL = pd.DataFrame({'Separations': seps_list,
                            'R-L (MHz)':RL_list})

# Include it as a column
Results['R-L (MHz)'] = Results_RL["R-L (MHz)"]

# Calculate the average from each scan, and add the column of average sepaations
avg_dist = (Results_RL["R-L (MHz)"] + Results["L-R (MHz)"])/2

Results['Average'] = avg_dist

# Add a column of the expected separations.  See the experiment instructions for values.
Results['Expected (MHz)'] = [1000, 9890, 5583]

# Show the completed table
Results

,Separations,L-R (MHz),R-L (MHz),Average,Expected (MHz)
0,2S1/2-3P3/2 to 2P1/2-3D3/2,(1.11+/-0.15)e+03,(1.08+/-0.14)e+03,(1.10+/-0.10)e+03,1000
1,2P3/2-3D5/2 to 2P1/2-3D3/2,(1.009+/-0.021)e+04,(1.013+/-0.024)e+04,(1.011+/-0.019)e+04,9890
2,2P3/2-3D5/2 to 2S1/2-3P1/2,(5.65+/-0.15)e+03,(5.74+/-0.19)e+03,(5.70+/-0.13)e+03,5583


## Present final values

The DataFrame tables have limited control over their formatting.  Assuming you have used the names provided, execute the following print commands to make a tbale with increased precision for comparison to expected results.

In [18]:
print('\n  Fine-structure line separation        |  Measured (MHz) |  Best known value (MHz)')
print('----------------------------------------|-----------------|-------------------------')
print('{:s} (Lamb shift) |   {:.2uP}       |   {:.0f}'.format(Results.iloc[0,0],Results['Average'][0],
                                                             Results['Expected (MHz)'][0]))
print('{:s}              |   {:.2uP}       |   {:.0f}'.format(Results.iloc[1,0],Results['Average'][1],
                                                             Results['Expected (MHz)'][1]))
print('{:s}              |   {:.2uP}       |   {:.0f}'.format(Results.iloc[2,0],Results['Average'][2],
                                                             Results['Expected (MHz)'][2]))


  Fine-structure line separation        |  Measured (MHz) |  Best known value (MHz)
----------------------------------------|-----------------|-------------------------
2S1/2-3P3/2 to 2P1/2-3D3/2 (Lamb shift) |   (1.10±0.10)×10³       |   1000
2P3/2-3D5/2 to 2P1/2-3D3/2              |   (1.011±0.019)×10⁴       |   9890
2P3/2-3D5/2 to 2S1/2-3P1/2              |   (5.70±0.13)×10³       |   5583
